In [45]:
import spacy
from spacy.tokens import Span
from spacy.matcher import PhraseMatcher

nlp = spacy.load("en_core_web_sm")
doc = nlp("I love coffee")

print(doc.text)
animals = ["Golden Retriever", "cat", "turtle", "Rattus norvegicus"]
matcher = PhraseMatcher(nlp.vocab)

I love coffee


In [53]:
def animal_component(doc):
    animals = ["Golden Retriever", "cat", "turtle", "Rattus norvegicus"]
    matcher = PhraseMatcher(nlp.vocab)
    
    matches = matcher(doc)
    span = [Span(doc, start, end, label="ANIMAL") for match_id, start, end in matches]
    doc.ents = span
    return doc

In [54]:
global nlp
nlp.add_pipe(animal_component, after=False)

time_spent = []

In [55]:
from time import time

def timer(func):
    def f(*args, **kwargs):
        global time_spent
        before = time()
        rv = func(*args, **kwargs)
        after = time()
        time_spent[func.__name__] = format(after - before, '.2f')# round(int(after - before), 3)
        return rv
    return f

In [56]:
@timer
def get_entities(doc):
    return [(ent.text, ent.label_) for ent in doc.ents]

@timer
def get_verbs(doc):
    return [(token.text, token.pos_) for token in doc if token.pos_ == "VERB" or token.pos_ == "AUX"]

@timer
def get_countries(doc):
    return [(ent.text, ent.label_) for ent in doc.ents if ent.label_ == 'GPE']

@timer
def get_persons(doc):
    return [(ent.text, ent.label_) for ent in doc.ents if ent.label_ == 'PERSON']


In [51]:
TEXT = """
I dont't wanna do it today, maybe tomorrow morning.
"""
doc = nlp(TEXT)


print('VERBS   :',  get_verbs(doc))
print('VERBS   :',  get_entities(doc))
print('VERBS   :',  get_countries(doc))
print('VERBS   :',  get_persons(doc))
print(time_spent)

NameError: name 'time_spent' is not defined

In [59]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

In [15]:
results = classifier(["We are very happy to show you the 🤗 Transformers library", "We hope you don't hate it."])

for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: POSITIVE, with score: 0.9998
label: NEGATIVE, with score: 0.5309


In [2]:
import csv

p = '../dataset/datasets_73978_166769_fb_sentiment.csv'
tweets = []
with open(p, newline='') as f:
    spamreader = csv.DictReader(f)
    for row in spamreader:
        tweets.append(row["FBPost"])
        
tweets[0]

'Drug Runners and  a U.S. Senator have something to do with the Murder http://www.amazon.com/Circumstantial-Evidence-Getting-Florida-Bozarth-ebook/dp/B004FPZ452/ref=pd_rhf_p_t_1 The State Attorney Knows... NOW So Will You. GET Ypur Copy TODAY'

In [43]:
import pandas as pd
import collections
from collections import Counter
import numpy as np

review_subset = pd.read_csv('/home/yanick/KRAKEN/PIP/NLP/data/Yelp Dataset/yelp.csv')

by_rating = collections.defaultdict(list)

for _, row in review_subset.iterrows():
    by_rating[row.stars].append(row.to_dict())

by_rating[1][0]

{'business_id': 'IJ0o6b8bJFAbG6MjGfBebQ',
 'date': '2010-09-05',
 'review_id': 'Dx9sfFU6Zn0GYOckijom-g',
 'stars': 1,
 'text': "U can go there n check the car out. If u wanna buy 1 there? That's wrong move! If u even want a car service from there? U made a biggest mistake of ur life!! I had 1 time asked my girlfriend to take my car there for an oil service, guess what? They ripped my girlfriend off by lying how bad my car is now. If without fixing the problem. Might bring some serious accident. Then she did what they said. 4 brand new tires, timing belt, 4 new brake pads. U know why's the worst? All of those above I had just changed 2 months before!!! What a trashy dealer is that? People, better off go somewhere!",
 'type': 'review',
 'user_id': 'zRlQEDYd_HKp0VS3hnAffA',
 'cool': 0,
 'useful': 1,
 'funny': 1}

In [60]:


results = classifier(" really I cant believe this place has received such high reviews from people\n\nmy lady and i walked in and were greeted rather rudely by a pretentious bitch at the front with a monotone name please instead of a warm friendly french welcome to this rather charming looking place we didnt have a reservation which from the looks of how dead empty the place was didnt seem like a problem until the hostess whisked through her reservation list and explained shed try to fit us in it was 6pm the place had one other couple dining and we were informed that the next party was arriving at 730 we werent really looking to hang out it would have be awfully lovely if yelpers would have informed of the need to MAKE RESERVATIONS before going in here because if you dont youre automatically a piece of shit according to the staff sorry i dont plan very much ahead next time i will and it wont be here\n\nanyways after debating for a minute or two with herself the hostess decided we could squeeze in and sat us right on top of the other couple inside the restaurant she informed us that we could cork our own wine for their low low price of 9 fucking dollars in words only office space could adequately describecoup des tartes what is it exactly that you do \n\nthe food was good nothing spectacular for 25 a plate we got a salad with apples and nuts and bleu cheese which was served warm now this is the first warm salad ive had all of my life and by no means am i some sort of wordly food expert but i believe salad should be cold and crisp not soggy and warm yuck the cordon bleu was tasty and filling however the wobbly table was an extreme annoyance when cutting through the chicken but it was solved with no thanks to the staff by placing a jackinthebox gift card under the leg i will admit the mashed potatoes are some of the creamiest and best ive ever had my lady got the pork tenderloin which had a strange pepperysweet pairing which i didnt particularly care for we were broke for desert so unfortunately we didnt get to try any tarts you would think for a place that promotes itself as being so poise would get some fancier menus other than a 15 piece of cardstock copied in bulk at kinkos\n\nthis place was rather disappointing i honestly thought it would be way more comfy and welcoming inside but it really wasnt it presents itself as being so high class and french and ohlala wee wee but when it comes down to it its still located in a tiny shack of a house at 16th street and highland maybe if it were located in the biltmore id give it a bit more cred ")
print("RESULT:  ", results)

IndexError: index out of range in self

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")

In [14]:
from transformers import pipeline

context = r"""
Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a
question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune
a model on a SQuAD task, you may leverage the examples/question-answering/run_squad.py script.
"""

nlp = pipeline("question-answering")
result = nlp(question="What is extractive question answering?", context=context)
print(result)

/home/yanick/anaconda3/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,


{'score': 0.6225793957710266, 'start': 34, 'end': 96, 'answer': 'the task of extracting an answer from a text given a question.'}


In [15]:
nlp = pipeline("ner")

In [3]:
from transformers import pipeline

summarizer = pipeline("summarization")

In [11]:
ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

#s = summarizer(ARTICLE, max_length=118, min_length=30, do_sample=False)
#print(s[0]["summary_text"])

In [5]:
ARTICLE = """
In order to analyze what has been done in Cape Verde with regard to Intellectual Property 
and also Digital Signature, we interviewed the competent entities related to each particular subject
and conducted interviews and applied questionnaires that we sent through electronic mails 
and social networks to our artists and authors both in Cape Verde and some who are living
abroad. With the data collected it was possible to verify that the protection of Intellectual
Property is still in a phase of maturation in our archipelago and having been given more
attention in recent years.
"""
#print(len(ARTICLE))

#print(summarizer(ARTICLE, max_length=118, min_length=30, do_sample=False))

In [25]:
import string
s = "@Maria: Do you mean the Nook? Be careful, books you buy for the Kindle are for that piece of electronics, and vice versa. I love my Kindle, there are people that swear by the Nook. They like the color screen.Me? I want an ereader that is a reader-- so I dont need color. The kindle battery lasts longer, and the unit isnt as heavy, which can make a difference after reading for a few hours. :) " # Sample string 
out = s.translate(str.maketrans('', '', string.punctuation))


doc = nlp(s.replace("@", ""))
print()
print('Entities:   ', get_entities(doc))
print('Countries   :',  get_countries(doc))
print('PERSON   :',  get_persons(doc))



Entities:    [('Maria', 'PERSON'), ('Kindle', 'PERSON'), ('a few hours', 'TIME')]
Countries   : []
PERSON   : [('Maria', 'PERSON'), ('Kindle', 'PERSON')]


In [2]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

In [3]:
def answer_question(question):
    inputs = tokenizer(question, ARTICLE, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]
    
    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer_start_scores, answer_end_scores = model(**inputs)
    
    answer_start = torch.argmax(
        answer_start_scores
    )
    answer_end = torch.argmax(answer_end_scores) + 1
    
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    return answer

In [8]:
answer_question("How we did to analyze what has been done?")

'the protection of intellectual property is still in a phase of maturation in our archipelago and having been given more attention in recent years'

In [40]:
import numpy as np
from annoy import AnnoyIndex

p = '../dataset/glove.6B.200d.txt'

class PreTrainedEmbeddings(object):
    def __init__(self, word_to_index, word_vectors):
        """
        Args:
            word (str)
        """
        self.word_to_index = word_to_index
        self.word_vectors = word_vectors
        self.index_to_word = { v: k for k, v in self.word_to_index.items() }
        self.index = AnnoyIndex(len(word_vectors[0]), metric='euclidean')
        
        for _, i in self.word_to_index.items():
            self.index.add_item(i, self.word_vectors[i])
        self.index.build(100)
        
    @classmethod
    def from_embeddings_file(cls, embedding_file):
        word_to_index = {}
        word_vectors = []
        with open(embedding_file) as fp:
            for line in fp.readlines():
                line = line.split(" ")
                word = line[0]
                vec = np.array([ float(x) for x in line[1:] ])
                
                word_to_index[word] = len(word_to_index)
                word_vectors.append(vec)
        return cls(word_to_index, word_vectors)
    
    def get_embedding(self, word):
        """
        Args:
            word (str)
        Returns:
            numpy.array
        """
        return self.word_vectors[self.word_to_index[word]]

    def get_closest_to_vector(self, vector, n=1):
        """Given a vector, return its n nearest neighbors
        Args:
            vector (np.array)
            n (int) -> the number of neighbors to return
        Returns:
            [str,str...] -> words nearest to the given vector
        """
        nn_indices = self.index.get_nns_by_vector(vector, n)
        return [self.index_to_word[neighbor] for neighbor in nn_indices]
    
    def compute_analogy(self, word1, word2, word3):
        """
        Args:
            word1 (str)
            word2 (str)
            word3 (str)
        Returns:
            closest_words (list) -> a list of nearest neighbors
        """
        vec1 = self.get_embedding(word1)
        vec2 = self.get_embedding(word2)
        vec3 = self.get_embedding(word3)
        
        # Symple hypothesis: Analogy is a spatial relationship
        spatial_relationship = vec2 - vec1
        vec4 = vec3 + spatial_relationship
        
        closest_words = self.get_closest_to_vector(vec4, n=4)
        existng_words = set([word1, word2, word3])
        closest_words = [word for word in closest_words if word not in existng_words]
        
        return closest_words

In [41]:
embeddings = PreTrainedEmbeddings.from_embeddings_file(p)

In [45]:
word1, word2, word3 = 'man', 'doctor', 'woman'
closest_words = embeddings.compute_analogy(word1, word2, word3)

print(closest_words)
for word4 in closest_words:
    print("{} : {} :: {} : {}".format(word1, word2, word3, word4))

['nurse', 'physician']
man : doctor :: woman : nurse
man : doctor :: woman : physician


In [36]:
closest_words = embeddings.compute_and_print_analogy('fast', 'fastest', 'young')

for word4 in closest_words:
    print("{} : {} :: {} : {}".format(word1, word2, word3, word4))

fast : fastest :: young : youngest
fast : fastest :: young : female
fast : fastest :: young : fellow


In [37]:
closest_words = embeddings.compute_and_print_analogy('talk', 'communicate', 'read')

for word4 in closest_words:
    print("{} : {} :: {} : {}".format(word1, word2, word3, word4))

talk : communicate :: read : interpret
talk : communicate :: read : typed
talk : communicate :: read : correctly
talk : communicate :: read : instructions
